In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

<ol><li>Загрузите данные, проверьте число наблюдений и столбцов, типы данных, наличие пропущенных значений, какие уникальные значения встречаются.</li>
<li>Постройте график распределения числа показов (Impressions) для каждой рекламы, прологарифмировав значения.</li>
<li>Создайте новую колонку c CTR. Посмотрите на описательные статистики и распределение.</li>
<li>Проанализируйте CTR с разбивкой по рекламной кампании.</li>
<li>Посчитайте стоимость за клик пользователя по объявлению (CPC). Изучите полученные значения, используя меры центральной тенденции и меры изменчивости.</li>
<li>Визуализируйте CPC с разбивкой по полу пользователей, которым были показаны объявления.</li>
<li>Посчитайте конверсию из клика в покупку.</li></ol>

<h4>Описание данных</h4>
<li><b>ad_id</b> - идентификатор объявления (рекламы)</li>
<li><b>xyz_campaign_id</b> – идентификатор рекламной кампании в базе компании X</li>
<li><b>fb_campaign_id</b> – идентификатор кампании в Facebook</li>
<li><b>age – возрастная</b> группа, которой показывалась реклама</li>
<li><b>gender</b> – пол тех, кому показывалась реклама</li>
<li><b>interest</b> –код, указывающий категорию, к которой относится интерес пользователя (соответствие число – интерес)</li>
<li><b>impressions</b> – число показов</li>
<li><b>clicks</b> – число кликов</li>
<li><b>spent –</b> сумма, которую компания потратила на показ объявления </li>
<li><b>total_conversion</b> – количество людей, которые заинтересовались продуктом после просмотра рекламы</li>
<li><b>approved_conversion</b> – количество людей, которые купили продукт после просмотра рекламы</li>

In [2]:
ad_data = pd.read_csv('conversion.csv')

In [3]:
ad_data.head()

,ad_id,xyz_campaign_id,fb_campaign_id,age,gender,interest,Impressions,Clicks,Spent,Total_Conversion,Approved_Conversion
0,708746,916,103916,30-34,M,15,7350,1,1.43,2,1
1,708749,916,103917,30-34,M,16,17861,2,1.82,2,0
2,708771,916,103920,30-34,M,20,693,0,0.00,1,0
3,708815,916,103928,30-34,M,28,4259,1,1.25,1,0
4,708818,916,103928,30-34,M,28,4133,1,1.29,1,1


In [4]:
ad_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ad_id                1143 non-null   int64  
 1   xyz_campaign_id      1143 non-null   int64  
 2   fb_campaign_id       1143 non-null   int64  
 3   age                  1143 non-null   object 
 4   gender               1143 non-null   object 
 5   interest             1143 non-null   int64  
 6   Impressions          1143 non-null   int64  
 7   Clicks               1143 non-null   int64  
 8   Spent                1143 non-null   float64
 9   Total_Conversion     1143 non-null   int64  
 10  Approved_Conversion  1143 non-null   int64  
dtypes: float64(1), int64(8), object(2)
memory usage: 98.4+ KB
